In [91]:
#Install all the library we are going to use in this exercise
import numpy as np
import pandas as pd
!conda install -c anaconda lxml --yes
import lxml.html as hl
!conda install -c conda-forge geocoder -y
import geocoder
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')
import re

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         156 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.6.16-py36_1  conda-forge --> 2019.6.16-py36_1 anaconda
    openssl: 1.1.1c-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0 anaconda


certifi-2019.6.16    | 156 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 

# Get the data from the wikipedia webpage 

In [92]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tableList = pd.read_html(url,header =0)
print("Done")

Done


In [93]:
firstTable = tableList[0]

In [94]:
firstTable.shape

(288, 3)

In [95]:
print(firstTable)

    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
283      M8Z         Etobicoke              Mimico NW
284      M8Z         Etobicoke     The Queensway West
285      M8Z         Etobicoke  Royal York South West
286      M8Z         Etobicoke         South of Bloor
287      M9Z      Not assigned           Not assigned

[288 rows x 3 columns]


## Sort the dataframe by the postcode in case they are not in order.

In [96]:
SortedT = firstTable.sort_values(by =["Postcode"]).reset_index(drop = True)

# Use for loop to go through the dataframe and perform the operations required by the assignment.

In [97]:
for Ind,PC,B,N in zip(SortedT.index.values,SortedT["Postcode"],SortedT["Borough"],SortedT["Neighbourhood"]):
    if B =="Not assigned":
        SortedT.drop(Ind, inplace = True) # delete from SortedT when "Borough" is "Not assigned" and go to next iteration
        continue
    if N == "Not assigned":
        SortedT.loc[Ind,"Neighbourhood"]=B # replace value in SortedT when "Neighbourhood" is "Not assigned"
    if PC == SortedT.loc[Ind +1,"Postcode"]:
        # if the next entry in SortedT has the same postcode, we will concentate the Neighbourhood from
        # this iteration to the next entry 
        SortedT.loc[Ind+1,"Neighbourhood"] = N +","+SortedT.loc[Ind+1,"Neighbourhood"]
        # Then delete this entry from the SortedT by drop(index)
        SortedT.drop(Ind, inplace = True)

In [98]:
SortedT.reset_index(drop = True)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"St. Phillips,Richview Gardens"
101,M9V,Etobicoke,"Jamestown,Beaumond Heights"


In [99]:
SortedT.shape

(103, 3)

# I tried to use geocoder to obtain coordinates of the neighbourhood, but it does not seems to be responding.

In [100]:
#Lat=[]
#Lon =[]

#for PC in SortedT["Postcode"]:
# initialize your variable to None
#    lat_lng_coords = None

# loop until you get the coordinates
#    while(lat_lng_coords is None):
#      g = geocoder.google('{}, Toronto, Ontario'.format(PC))
#      lat_lng_coords = g.latln
#    Lat.append(lat_lng_coords[0])
#    Lon.append(lat_lng_coords[1])

# Read coordinate data from CSV instead and merge the 2 dataframes

In [101]:
csv_url = "http://cocl.us/Geospatial_data"
Geo_pd = pd.read_csv(csv_url)

In [102]:
Geo_pd.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [103]:
Geo_pd.shape

(103, 3)

# Join the 2 dataframe together by matching their Postal Code

In [104]:
toronto_coor = pd.merge(SortedT,Geo_pd,left_on="Postcode",right_on="Postal Code")

In [105]:
toronto_coor.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [106]:
toronto_coor.shape

(103, 6)

# Get the map of Toronto and mark the locations of neighborhoods on them.

In [107]:
address = 'Toronto'

geolocator = Nominatim(user_agent="T_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [108]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_coor['Latitude'], toronto_coor['Longitude'], toronto_coor['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [109]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [110]:
neighborhood_latitude = toronto_coor.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_coor.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_coor.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern,Rouge are 43.806686299999996, -79.19435340000001.


In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

# Test out the the request with 1st Neighbourhood on the list.

In [ ]:
results = requests.get(url).json()
results

# Define a function that uses Foursquare API to get the top 100 locations near each neighbourhood.

In [113]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                                 'Neighborhood Latitude', 
                                 'Neighborhood Longitude', 
                                 'Venue', 
                                 'Venue Latitude', 
                                 'Venue Longitude', 
                                 'Venue Category']
    
    return(nearby_venues)
         
     

In [114]:
toronto_venues =getNearbyVenues(names =toronto_coor['Neighbourhood'] ,
                                latitudes=toronto_coor['Latitude'],
                                longitudes = toronto_coor['Longitude'])
                      

Malvern,Rouge
Rouge Hill,Port Union
Morningside,Guildwood
Woburn
Cedarbrae
Scarborough Village
Ionview,East Birchmount Park
Golden Mile,Clairlea
Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Scarborough Town Centre,Wexford Heights
Wexford,Maryvale
Agincourt
Clarks Corners,Sullivan
L'Amoreaux East,Agincourt North
L'Amoreaux West
Upper Rouge
Hillcrest Village
Henry Farm,Fairview
Bayview Village
Silver Hills,York Mills
Willowdale,Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South,Flemingdon Park
Bathurst Manor,Wilson Heights
York University,Northwood Park
Downsview East,CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill,Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
India Bazaar,The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Rathnelly,Deer P

In [115]:
toronto_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern,Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Morningside,Guildwood",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Morningside,Guildwood",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Morningside,Guildwood",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


# After calling the getNearbyVenues function, the number of rows of neighborhood reduced by 4.
# This is probably due to empty results returned by the explore URL on some neighborhood.

In [116]:
toronto_venues["Neighborhood"].nunique()

99

# For my exploration of data, I want to see which 10 neighborhoods have the highest numbers of restaurants within a 500 meters radius.

# 1. Use Regex to search for venues classifiied as restaurant and filter out all the entries that are not restaurant.

In [117]:

booleans = []
for venue in toronto_venues['Venue Category']:
    if re.search('Restaurant', venue):
        booleans.append(True)
    else:
        booleans.append(False)
Filter = pd.Series(booleans)   
Restaurant_Neighbourhood = toronto_venues[Filter]
Restaurant_Neighbourhood.shape    

(524, 7)

# Group the restaurant venues by neighborhood, count the entries in the group and sort them in ascending order.
# Then get the top 10 neighborhoods with the highest number of restaurant within a 500 meters radius.

In [118]:
Restaurant_Neighbourhood.groupby('Neighborhood').count()[['Venue']].sort_values(by = 'Venue',ascending = False).head(10)

,Venue
Neighborhood,
"Kensington Market,Chinatown",34
Church and Wellesley,31
"King,Richmond",27
"Commerce Court,Victoria Hotel",27
"First Canadian Place,Underground city",27
St. James Town,26
Central Bay Street,25
"Toronto Dominion Centre,Design Exchange",23
"Trinity,Little Portugal",22


# For clustering of data, I will do one hot encoding to convert values in Venues category into sets of features to be use for K mean clustering.

In [119]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern,Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Morningside,Guildwood",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Morningside,Guildwood",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Morningside,Guildwood",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Bathurst Manor,Wilson Heights",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0
2,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.017857,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Birch Cliff,Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
95,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
96,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0
97,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 2, 1, 1, 1], dtype=int32)

In [122]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [123]:
toronto_grouped.head()

,Cluster Labels,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,"Bathurst Manor,Wilson Heights",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.055556,0.0,0.0,0.0,0.0,0.0,0.0
2,1,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,1,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.017857,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,1,"Birch Cliff,Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
toronto_merged = toronto_coor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_grouped.set_index('Neighbourhood'), on='Neighbourhood',rsuffix="T.")

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,Accessories Store,Afghan Restaurant,Airport,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,Scarborough,"Malvern,Rouge",M1B,43.806686,-79.194353,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,Scarborough,"Rouge Hill,Port Union",M1C,43.784535,-79.160497,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,Scarborough,"Morningside,Guildwood",M1E,43.763573,-79.188711,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# If you look at the result returned by the Kmean.fit(), you will see that the numbers of rows reduced by 4.
# This is probably due to some neighborhoods returning empty result with the explore URL.

In [125]:
toronto_merged.shape

(103, 284)

In [126]:
toronto_grouped.shape

(99, 279)

# Filter out the neighborhood without any cluster labels.


In [127]:
TestNull = toronto_merged['Cluster Labels'].notna()
TestNull

0      True
1      True
2      True
3      True
4      True
       ... 
98     True
99     True
100    True
101    True
102    True
Name: Cluster Labels, Length: 103, dtype: bool

In [128]:
DfMapTorCluster = toronto_merged[TestNull]

# Create map to visualise the clustering of neighbourhood

In [129]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DfMapTorCluster['Latitude'], DfMapTorCluster['Longitude'], DfMapTorCluster['Neighborhood'], DfMapTorCluster['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters